#### 감정 분석 자연어 처리 
1. data 폴더 안에 ratings_train.txt 파일을 로드 
2. 데이터를 상위 500개 데이터만 추출
    - 데이터를 25000부터 30000 번째의 데이터를 이용
3. 리뷰 데이터와 감정 데이터로 나눠준다. 
4. 리뷰 데이터를 토큰화(komoran함수 이용)
5. Word2Vec 학습
    - window -> 5
    - epochs -> 100
    - min_count -> 2
    - sg -> 1
    - seed -> 42
6. 벡터화(Word2Vec, 단위 벡터의 평균)
7. 분류 모델 ( SVC, Logistic )
8. train, test을 이용하여 2개의 모델중 성능이 높은 모델이 무엇인가?
9. 단위 백터의 평균의 성능과 단위 벡터 + 중요도 평균의 성능의 차이를 확인


In [ ]:
import numpy as np 
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [ ]:
df = pd.read_csv('../data/ratings_train.txt', sep='\t')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# 상위 데이터 500개 
# df2 = df.head(500)
df2 = df.loc[:500, ]

In [ ]:
df2['label'].value_counts()

In [ ]:
# 토큰화 함수 생성 
# komoran 사용 (konlpy 설치가 되어있는 경우)
# 설치가 되어있지 않은 경우에는 split()을 이용하여 토큰화 
def build_tokenize():
    try:
        # 라이브러리 로드 -> 라이브러리가 존재하면 코드들 실행 
        from konlpy.tag import Komoran
        komoran = Komoran()
        allow_pos = ['NNP', 'NNG', 'VV', 'VA', 'SL', 'MAG']
        def tokenize(text):
            tokens = []
            for word, pos in komoran.pos(text):
                if pos in allow_pos:
                    tokens.append(word)
            return tokens
        # tokenize 함수를 결과로 되돌려준다. 
        return tokenize
    except Exception as e:
        print("Komoran 사용 불가 : ", e)
        return lambda x : x.split()


In [ ]:
tokenize = build_tokenize()

In [ ]:
reviews = df2['document'].values
Y = df2['label'].values

In [ ]:
X_tokens = [ tokenize(review) for review in reviews]

In [ ]:
# Word2Vec을 이용하여 학습(Skip-gram 방식)
w2v = Word2Vec(
    sentences= X_tokens, 
    window = 5, 
    min_count= 2, 
    sg = 1, 
    epochs= 100, 
    seed = 42
)

wv = w2v.wv

In [ ]:
def sent_embed_mean(tokens):
    vecs = []
    for word in tokens:
        if word in wv.index_to_key:
            vecs.append(wv[word])
    result = np.mean(vecs, axis=0) if vecs else np.zeros(wv.vector_size)
    return result


In [ ]:

# 단어별 중요도 
tfidf_vec = TfidfVectorizer(
    tokenizer= tokenize, 
    lowercase= False
).fit(reviews)
idf = dict(
    zip(
        # get_feature_names_out() -> Tfidf에서 사용된 단어들의 목록
        tfidf_vec.get_feature_names_out(), 
        # idf_ : 중요도
        tfidf_vec.idf_
    )
)
# 단어 별 단위 벡터의 평균과 idf을 곱한다. 
def sent_embed_tfidf(tokens):
    vecs = []
    weight = []
    for word in tokens:
        # tokens에 각각의 단어가 Word2Vec과 TF-IDF에 존재한다면
        if word in wv.key_to_index and word in idf:
            # vecs -> 단위벡터와 중요도를 곱한 값을 vecs 추가
            vecs.append(wv[word] * idf[word])
            # weight -> 중요도 데이터를 추가 
            weight.append(idf[word])
    # vecs의 데이터가 존재하지 않는다면 -> tokens 안에 단어는 존재하지만 Word2Vec이나
    # TD-IDF에 단어가 존재하지 않을때
    if not vecs:
        # 희소 행렬 되돌려준다. 0행렬
        result = np.zeros(wv.vector_size)
    else:
        result = np.sum(vecs, axis=0) / ( np.sum(weight) + 1e-9 )
    return result


In [ ]:
X_embed = [ sent_embed_mean(token) for token in X_tokens ]
X_embed

In [ ]:
X_embed2 = [ sent_embed_tfidf(token) for token in X_tokens ]
X_embed2

In [ ]:
# 모델 2개 객체 생성 
svc = SVC(random_state= 42)
logi = LogisticRegression(random_state=42)

In [ ]:
def run_model(X, Y, model, test_size = 0.2):
    # X는 독립 변수
    # Y는 종속 변수
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size= test_size, random_state=42, stratify=Y
    )
    # 모델에 학습
    model.fit(X_train, Y_train)
    # 학습된 모델에 예측 값
    y_pred = model.predict(X_test)
    print("정확도 : ", round(
        accuracy_score(y_pred, Y_test), 4
    ))
    print('분류 레포드 : ')
    print(classification_report(y_pred, Y_test))


In [ ]:
run_model(X_embed, Y, svc)

In [ ]:
run_model(X_embed, Y, logi)

In [ ]:
run_model(X_embed2, Y, svc)

In [ ]:
run_model(X_embed2, Y, logi)

- df에서 하위 10개 데이터를 이용하여 예측 

In [ ]:
# 학습된 모델에 예측의 값을 반환하는 함수 
# 세번째 매개변수(vec_type)를 생성 -> 기본값은 'mean'
# 'tfidf' 입력이 들어온다면 벡터화 작업은 w2v + ifidf 융합한 벡터화 
def predict_sentence_list(sentences, model, vec_type = 'mean'):
    # sentences : 문장들의 리스트 
    # 문장들을 토큰화 -> 임베딩 
    X_test = []
    for sent in sentences:
        # token() 함수를 호출하여 토큰화 
        tokens = tokenize(sent)
        # 토큰화 된 문장을 sent_enbed_mean 함수에 입력하여 호출 ( 단위 백터의 평균 )
        if vec_type == 'mean':
            vec = sent_embed_mean(tokens)
        elif vec_type == 'tfidf':
            vec = sent_embed_tfidf(tokens)
        X_test.append(vec)
    
    preds = model.predict(X_test)
    result = []
    for sent, pred in zip(sentences, preds):
        label = "긍정" if pred == 1 else "부정"
        result.append([sent, label])
    return result



In [ ]:
# 모델 학습 -> 예측
X_test = df['document'].tail(10).values
run_model(X_embed, Y, svc)

predict_sentence_list(X_test, svc, vec_type='mean')

In [ ]:
run_model(X_embed2, Y, logi)
predict_sentence_list(X_test, logi, vec_type='tfidf')